In [1]:
import pandas as pd
import numpy as np

In [3]:
#Se especifican los tipo de datos para mejorar la performance
eventos = pd.read_csv('../dataTP2/events_up_to_01062018.csv', dtype={"timestamp": object,#intenté hacerla datetime y no anduvo 
                                                                     "event": 'category',
                                                                     "person":object,
                                                                     "url":object,
                                                                     "sku":object,#intenté ponerle int y no anduvo, con float si, pero finalmente me di cuenta que no tiene sentido
                                                                     "model":object,
                                                                     "condition":'category',
                                                                     "storage":object,
                                                                     "color":'category',
                                                                     "skus":object,
                                                                     "search_term":object,
                                                                     "staticpage":object,
                                                                     "campaign_source":object,
                                                                     "search_engine":object,
                                                                     "channel":object,
                                                                     "new_vs_returning":'category',
                                                                     "city":object,
                                                                     "region":object,
                                                                     "country":object,
                                                                     "device_type":object,
                                                                     
                                                                    })
etiquetas = pd.read_csv('../dataTP2/labels_training_set.csv',dtype={"person":object, "label":bool})

/home/nico/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
etiquetas.head(6)

,person,label
0,0566e9c1,False
1,6ec7ee77,False
2,abe7a2fb,False
3,34728364,False
4,87ed62de,False
5,db2c4d27,True


In [5]:
etiquetas['label'].value_counts()

False    18434
True       980
Name: label, dtype: int64

In [6]:
eventos.dtypes

timestamp                     object
event                       category
person                        object
url                           object
sku                           object
model                         object
condition                   category
storage                       object
color                       category
skus                          object
search_term                   object
staticpage                    object
campaign_source               object
search_engine                 object
channel                       object
new_vs_returning            category
city                          object
region                        object
country                       object
device_type                   object
screen_resolution             object
operating_system_version      object
browser_version               object
dtype: object

In [7]:
eventos.head(10)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2018-05-18 00:44:27,searched products,4c8a8b93,NaN,NaN,NaN,NaN,NaN,NaN,"10240,9987,10322,10085,9944,9931,13404,10154,1...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2018-05-18 00:44:14,viewed product,1b9f7cf6,NaN,2831.0,iPhone 6,Bom,16GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2018-05-18 00:44:02,viewed product,29ebb414,NaN,2845.0,iPhone 6 Plus,Bom,128GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018-05-18 00:43:59,viewed product,de8fe91b,NaN,12548.0,Motorola Moto G5 Plus,Bom,32GB,Platinum,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2018-05-18 00:43:40,ad campaign hit,45baf068,/,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
len(eventos['person'].unique())

38829

In [9]:
etiquetas.count()

person    19414
label     19414
dtype: int64

In [10]:
print(etiquetas['label'].value_counts())
print(etiquetas['label'].value_counts(normalize=True))

False    18434
True       980
Name: label, dtype: int64
False    0.949521
True     0.050479
Name: label, dtype: float64


In [11]:
eventos['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [15]:
#Nos quedamos con los eventos de los que tenemos idea de si convirtieron luego
eventosConocidos = eventos.loc[eventos['person'].isin(etiquetas['person'])]

In [16]:
eventosConocidos['event'].value_counts()

viewed product       629233
brand listing        105993
visited site         102299
ad campaign hit       94998
generic listing       80453
searched products     63887
search engine hit     52829
checkout              32539
staticpage             5660
conversion             3564
lead                    431
Name: event, dtype: int64

In [17]:
#De los eventos conocidos hago un merge. Claramente podría haber hecho el merge antes, pero cuando lo probé antes, falló por 
eventosConocidosMrg = eventosConocidos.merge(etiquetas,on='person')

In [18]:
eventosConocidosGrp = eventosConocidosMrg.groupby('person')

In [19]:
eventosConocidosGrp.head(3)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.0,iPhone 5s,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,5907.0,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
65,2018-05-18 00:44:14,viewed product,1b9f7cf6,NaN,2831.0,iPhone 6,Bom,16GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
66,2018-05-18 00:43:55,viewed product,1b9f7cf6,NaN,2830.0,iPhone 6,Bom,16GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
67,2018-05-18 00:44:40,viewed product,1b9f7cf6,NaN,2834.0,iPhone 6,Bom,64GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
90,2018-05-18 00:43:59,viewed product,de8fe91b,NaN,12548.0,Motorola Moto G5 Plus,Bom,32GB,Platinum,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
91,2018-05-18 00:36:03,search engine hit,de8fe91b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
92,2018-05-18 00:44:48,viewed product,de8fe91b,NaN,12507.0,Motorola Moto G5,Muito Bom,32GB,Ouro,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
143,2018-05-18 00:43:40,ad campaign hit,45baf068,/,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [20]:
len(eventosConocidosMrg.groupby('person').mean())


19414

In [21]:
len(eventos['sku'].unique())

3694

In [22]:
eventosConocidos.describe()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,1171886,1171886,1171886,94875,665336,665767,665336,665336,665336,249587,...,52829,102299,102299,102299,102299,102299,102299,102297,102299,102299
unique,857447,11,19414,236,3405,198,5,8,63,34843,...,4,7,2,1854,84,39,4,285,107,308
top,2018-05-31 00:50:28,viewed product,ffee0f18,/,2830.0,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",...,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,11,629233,3458,31753,4044,52268,275492,222134,160397,1338,...,52242,45632,83173,18569,28597,99160,53171,37827,22613,28256


In [ ]:
"""agregando features
viewed product       629233
brand listing        105993
visited site         102299
ad campaign hit       94998
generic listing       80453
searched products     63887
search engine hit     52829
checkout              32539
staticpage             5660
conversion             3564
lead                    431"""
eventosConocidos['viewedProduct'] = (eventos['event'].str.contains(''))

In [18]:
len(eventosConocidos['event'] == 'viewed product')

1171886

In [23]:
eventosConocidosMrg['viewedPrd'] = (eventosConocidosMrg['event'] == 'viewed product')
print(eventosConocidosMrg.groupby('person').sum())

          label  viewedPrd
person                    
0008ed71    0.0        0.0
000c79fe    0.0        3.0
001802e4    0.0        4.0
0019e639    0.0      189.0
001b0bf9    0.0        2.0
001ca5ee    0.0       52.0
001dfc31    0.0       18.0
001e9aea    0.0       17.0
0020152e    0.0        2.0
002b0188    0.0       12.0
002e74b2    0.0        2.0
00317e49    0.0        4.0
0031c75f    0.0       26.0
003476c0   43.0        6.0
003847fc    0.0       32.0
003a7d49    0.0        3.0
003bdb4d    0.0        1.0
003d4cac    8.0        2.0
003dd965    5.0        0.0
0042421c   22.0       10.0
004b8d91    0.0       10.0
004ed8ba    0.0       71.0
0050d971    0.0        4.0
00546e55    0.0        3.0
00665dda    0.0       15.0
006671fd   18.0       15.0
00702e02    0.0       55.0
00736d10    0.0       93.0
007537e8    0.0        6.0
007ddac1    0.0        2.0
...         ...        ...
ff8a9723    0.0       36.0
ff8af29b    0.0        1.0
ff906750    0.0        5.0
ff97d56f    0.0       79.0
f